In [1]:
import telebot
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler
from aiogram import types
import nest_asyncio
import logging
import random
import time
import urllib.request
nest_asyncio.apply()
import json
import numpy as np
from datetime import date
import datetime
from aiogram.dispatcher.filters import Text
from aiogram.dispatcher import FSMContext
from aiogram import Bot, Dispatcher, executor, types
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from aiogram.dispatcher.filters.state import State, StatesGroup
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove
import aiogram.utils.markdown as md
import mysql.connector as mariadb
from mysql.connector import Error
bot = Bot(token="5771726278:AAEmaEDz1ihWnAwpjXyvI8iTKNIFYd6J_ho")
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)
logging.basicConfig(level=logging.INFO)
connection = mariadb.connect(user = 'pyuser', password = 'i8Tj9in73c', database = 'psyanima', host = 'localhost', port = '4727')
curs = connection.cursor(buffered=True)
class Feelings(StatesGroup):
    answ = State()
    
class Replace(StatesGroup):
    repl = State()
    
global flag 
flag = 0

@dp.message_handler(commands="start")
async def cmd_start(message: types.Message): 
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Да!", "Нет, хочу поскорее приступить к работе."]
    keyboard.add(*buttons)
    await message.answer('Привет! Меня зовут Анима. Я — твой куратор в сфере психологии. Хочешь побольше узнать обо мне и моих возможностях?', reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Да!")
async def intro(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Все понятно, давай перейдем к делу."]
    keyboard.add(*buttons)
    await message.answer("😺В связи с популяризацией разных тематических чатов, многие сферы нашей жизни цифровизировались. В том числе и психология. \n\n😸Существует множество онлайн-сервисов по подбору психолога и поиску психологической помощи, большинство из них являются страницами в интернете. Однако, гораздо удобнее будет иметь чат прямо под рукой, например, в мессенджере. \n\n😼Так я и появилась! Добро пожаловать в чат, где ты сможешь найти различные упражнения, техники, медитации и многое другое, чтобы достичь единения с внутренним миром, либо же прокачать навыки в психологии. \n\nНачнем работу?🪄", reply_markup=keyboard)    

@dp.message_handler(lambda message: message.text in ["Все понятно, давай перейдем к делу.", "Нет, хочу поскорее приступить к работе.", "Назад"])
async def options(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Дневник самочувствия", "Упражнения", "Медитации", "Ежедневный инсайт", "Узнать больше о проекте"]
    keyboard.add(*buttons)
    await message.answer("Какой раздел тебя интересует?", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Дневник самочувствия")
async def diary(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Приступаем!", "Посмотреть записи за предыдущие дни", "Назад"]
    keyboard.add(*buttons)
    await message.answer("В этом разделе ты ежедневно можешь оставлять заметки о текущем дне или твоем самочувствии. Пиши все, что угодно, даже твой сегодняшний завтрак годится!", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Приступаем!")
async def diary_wr(message: types.Message):
    today = date.today()
    dt = today.strftime("%d%m%Y")
    curs.execute(f"SELECT * FROM psyanima.feels WHERE UserId = {message.from_user.id} AND Date = {dt}")
    if curs.fetchone() != None:
        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
        buttons = ["Да, хочу изменить запись.", "Назад"]
        keyboard.add(*buttons)
        await message.answer("Сегодня ты уже заполнял дневник. Желаешь изменить что-нибудь?", reply_markup=keyboard)  
    else:
        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
        buttons = ["Пропустить описание"]
        keyboard.add(*buttons)
        await Feelings.answ.set()
        await message.answer("Опиши, как ты себя чувствуешь? Можешь рассказать про свой день, что хорошего или плохого было сегодня?", reply_markup=keyboard)  

@dp.message_handler(lambda message: message.text == "Да, хочу изменить запись.")
async def ch_wr(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Пропустить описание"]
    keyboard.add(*buttons)
    await Replace.repl.set()
    await message.answer("Опиши, как ты себя чувствуешь? Можешь рассказать про свой день, что хорошего или плохого было сегодня?", reply_markup=keyboard) 

@dp.message_handler(state=Replace.repl)
async def wr(message: types.Message, state: FSMContext):
    async with state.proxy() as data:
        data['text'] = message.text
        user_message = data['text']
        today = date.today()
        dt = today.strftime("%d%m%Y")
        query = f"UPDATE psyanima.feels SET Value = '{user_message}' WHERE (UserID) = ({message.from_user.id}) AND Date = '{dt}'"
        curs.execute(query)
        connection.commit()
    await state.finish()
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["1", "2", "3", "4", "5"]
    global flag
    flag = 1
    keyboard.add(*buttons)
    if user_message == "Пропустить описание": await message.answer("Как бы ты оценил сегодняшний день по шкале от 1 до 5?", reply_markup=keyboard)
    else: await message.answer("Спасибо, что доверяешь мне! Как бы ты оценил сегодняшний день по шкале от 1 до 5?", reply_markup=keyboard)
    
@dp.message_handler(state=Feelings.answ)
async def grad(message: types.Message, state: FSMContext):
    async with state.proxy() as data:
        data['text'] = message.text
        user_message = data['text']
        query = f"SELECT COUNT(*) FROM psyanima.feels"
        curs.execute(query)
        ident = curs.fetchone()[0]
        query = f"INSERT INTO psyanima.feels (UserID, ID) VALUES ('{message.from_user.id}', '{ident}')"
        curs.execute(query)
        today = date.today()
        dt = today.strftime("%d%m%Y")
        query = f"UPDATE psyanima.feels SET Value = '{user_message}' WHERE (UserID) = ({message.from_user.id}) AND ID = '{ident}'"
        curs.execute(query)
        query = f"UPDATE psyanima.feels SET Date = '{dt}' WHERE (UserID) = ({message.from_user.id}) AND ID = '{ident}'"
        curs.execute(query)
        connection.commit()
    await state.finish()
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["1", "2", "3", "4", "5"]
    global flag
    flag = 1
    keyboard.add(*buttons)
    if user_message == "Пропустить описание": await message.answer("Как бы ты оценил сегодняшний день по шкале от 1 до 5?", reply_markup=keyboard)
    else: await message.answer("Спасибо, что доверяешь мне! Как бы ты оценил сегодняшний день по шкале от 1 до 5?", reply_markup=keyboard)
        
@dp.message_handler(lambda message: message.text in ["1", "2", "3", "4", "5"])
async def grad2(message: types.Message):
    global flag
    if flag == 1:
        flag = 0
        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
        buttons = ["Назад"]
        keyboard.add(*buttons)
        today = date.today()
        dt = today.strftime("%d%m%Y")
        query = f"UPDATE psyanima.feels SET Grade = '{message.text}' WHERE (UserID) = ({message.from_user.id}) AND Date = '{dt}'"
        curs.execute(query)
        connection.commit()
        if message.text <= "3": await message.answer("Мне искренне жаль, что этот день был не таким хорошим, как хотелось бы. Просто знай: я всегда рядом😉", reply_markup=keyboard)
        else: await message.answer("Я рада, что сегодня все было так хорошо!🥰", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Посмотреть записи за предыдущие дни")
async def last(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Сегодня", "Прошлая неделя", "Все записи", "Назад"]
    keyboard.add(*buttons)
    await message.answer("Желаешь посмотреть записи за последний день, неделю или все записи?", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Сегодня")
async def tod(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    today = date.today()
    dt = today.strftime("%d%m%Y")
    query = f"SELECT * FROM psyanima.feels WHERE UserId = {message.from_user.id} AND Date = {dt}"
    try:
        curs.execute(query)
        tmp = curs.fetchone()
        dt =datetime.datetime.strptime(dt,"%d%m%Y")
        dt = dt.strftime("%d.%m.%Y")
        await message.answer(f"Данные за {dt}\nТвоя запись: {tmp[2]}\nТвоя оценка: {tmp[3]}", reply_markup=keyboard)
    except:await message.answer(f"Не могу найти твои записи. Как насчет создания новой?", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Прошлая неделя")
async def week(message: types.Message):
    posts = []
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    for i in range(7, -1, -1):
        today = date.today() -  datetime.timedelta(days=i)
        dt = today.strftime("%d%m%Y")
        query = f"SELECT * FROM psyanima.feels WHERE UserId = {message.from_user.id} AND Date = '{dt}'"
        try:
            curs.execute(query)
            tmp = curs.fetchone()
            if tmp != None:
                posts.append(tmp)
        except:
            pass
    if len(posts) > 0:
        mes = ""
        for i in range(len(posts)):
            tmp = posts[i]
            today = datetime.datetime.strptime(tmp[1], '%d%m%Y')
            dt = today.strftime("%d.%m.%Y")
            mes += f"Данные за {dt}\nТвоя запись: {tmp[2]}\nТвоя оценка: {tmp[3]}\n\n"
        await message.answer(mes, reply_markup=keyboard)
    else: await message.answer(f"Не могу найти твои записи. Как насчет создания новой?", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Все записи")
async def all_m(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    query = f"SELECT * FROM psyanima.feels WHERE UserId = {message.from_user.id}"
    try:
        curs.execute(query)
        tmp = curs.fetchall()
    except:
        pass
    if len(tmp) > 0:
        mes = ""
        for i in range(len(tmp)):
            today = datetime.datetime.strptime(tmp[i][1], '%d%m%Y')
            dt = today.strftime("%d.%m.%Y")
            mes += f"Данные за {dt}\nТвоя запись: {tmp[i][2]}\nТвоя оценка: {tmp[i][3]}\n\n"
        await message.answer(mes, reply_markup=keyboard)
    else: await message.answer(f"Не могу найти твои записи. Как насчет создания новой?", reply_markup=keyboard)
    
@dp.message_handler(lambda message: message.text == "Упражнения")
async def exer(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Когнитивные способности", "Равновесие и координацию", "Воображение", "Вкусовое и обонятельное восприятие", "Слух", "Назад"]
    keyboard.add(*buttons)
    await message.answer("В этом разделе ты найдешь разные упражнения, большинство из которых не требуют никаких дополнительных предметов. Что ты хочешь развить?", reply_markup=keyboard)
    
@dp.message_handler(lambda message: message.text == "Когнитивные способности")
async def exer_c(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(0, 9)
    query = f"SELECT Exer FROM psyanima.exercise WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchall()
    await message.answer(f"Вот твое упражнение: \n\n{tmp[0][0]}\n\n Уверена, тебе понравится! Приятного времяпрепровождения🥰", reply_markup=keyboard)
                         
@dp.message_handler(lambda message: message.text == "Равновесие и координацию")
async def exer_r(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(10, 20)
    query = f"SELECT Exer FROM psyanima.exercise WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchall()
    await message.answer(f"Вот твое упражнение: \n\n{tmp[0][0]}\n\n У тебя все получится! Приятного времяпрепровождения🥰", reply_markup=keyboard)
    
@dp.message_handler(lambda message: message.text == "Воображение")
async def exer_i(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(21, 30)
    query = f"SELECT Exer FROM psyanima.exercise WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchall()
    await message.answer(f"Вот твое упражнение: \n\n{tmp[0][0]}\n\n Расслабся и представляй! Приятного времяпрепровождения🥰", reply_markup=keyboard)
   
@dp.message_handler(lambda message: message.text == "Вкусовое и обонятельное восприятие")
async def exer_v(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(31, 38)
    query = f"SELECT Exer FROM psyanima.exercise WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchall()
    await message.answer(f"Вот твое упражнение: \n\n{tmp[0][0]}\n\n Вкусных тебе запахов! Приятного времяпрепровождения🥰", reply_markup=keyboard)
    
@dp.message_handler(lambda message: message.text == "Слух")
async def exer_s(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(39, 43)
    query = f"SELECT Exer FROM psyanima.exercise WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchall()
    await message.answer(f"Вот твое упражнение: \n\n{tmp[0][0]}\n\n Напряги свой острый слух! Приятного времяпрепровождения🥰", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Медитации")
async def exer_m(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Расслабление", "Энергия", "Снятие стресса", "Назад"]
    keyboard.add(*buttons)
    await message.answer(f"Добро пожаловать в раздел медитаций. \n\nМедитация — это психическое упражнение, направленное на достижение внутреннего состояния спокойствия ума и тела при полной осознанности и внимательности. \n\nЗагрузка занимает некоторое время! Какую медитацию ты хочешь послушать?", reply_markup=keyboard)
    
@dp.message_handler(lambda message: message.text == "Расслабление")
async def exer_re(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(0,4)
    query = f"SELECT * FROM psyanima.meditation WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchone()
    await bot.send_voice(message.chat.id, open(f"{tmp[0]}.mp3", "rb"), caption = f"Автор медитации: {tmp[2]}\n\nНазвание медитации: {tmp[1]}", reply_markup=keyboard)
 
@dp.message_handler(lambda message: message.text == "Энергия")
async def exer_e(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(5,9)
    query = f"SELECT * FROM psyanima.meditation WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchone()
    await bot.send_voice(message.chat.id, open(f"{tmp[0]}.mp3", "rb"), caption = f"Автор медитации: {tmp[2]}\n\nНазвание медитации: {tmp[1]}", reply_markup=keyboard)
    
@dp.message_handler(lambda message: message.text == "Снятие стресса")
async def exer_re(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    rid = random.randint(10, 14)
    query = f"SELECT * FROM psyanima.meditation WHERE ID = {rid} AND Type = '{message.text}'"
    curs.execute(query)
    tmp = curs.fetchone()
    await bot.send_voice(message.chat.id, open(f"{tmp[0]}.mp3", "rb"), caption = f"Автор медитации: {tmp[2]}\n\nНазвание медитации: {tmp[1]}", reply_markup=keyboard)

@dp.message_handler(lambda message: "c56a9ab55f597cf928a7569a179957d8" in message.text)            
async def admin(message: types.Message):
    if message.from_user.id == 473283997:
        keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
        buttons = ["Назад"]
        keyboard.add(*buttons)
        query = f"SELECT UserId FROM psyanima.feels"
        curs.execute(query)
        tmp = np.unique(curs.fetchall())
        mes = message.text.replace("c56a9ab55f597cf928a7569a179957d8", "")
        for i in tmp: await bot.send_message(i, f"{mes}", reply_markup = keyboard) 
        await message.answer(f"Готово!", reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "Узнать больше о проекте")
async def proj(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    await message.answer("Данный бот был создан на конкурс Большая перемена.Профи, кейс от Волонтеров-медиков. \n\nАвтор — @suiremon (Тросько Виктория).\n\nПо всем вопросам, пожеланиям и предложениям пишите на почту suiremon@yandex.ru. \n\nСпасибо за использование!", reply_markup=keyboard)
 
@dp.message_handler(lambda message: message.text == "Ежедневный инсайт")
async def ins(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Назад"]
    keyboard.add(*buttons)
    today = date.today()
    dt = today.strftime("%d%m%Y")
    query = f"SELECT Advice FROM psyanima.daily_a WHERE Date = {dt}"
    curs.execute(query)
    tmp = curs.fetchall()
    await message.answer(f"Твой инсайт на сегодня:\n\n{tmp[0][0]}", reply_markup=keyboard)
    
if __name__ == "__main__":
    executor.start_polling(dp)